<h3>Split dataset</h3>

In [ ]:
from torchvision.datasets import MNIST
import numpy as np
from PIL import Image

train_target_size = 20000

class custum_MNIST(MNIST):

    def __init__(self, target, num, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.target = target
        if self.train:
            if target:
                # training data for target model
                self.data = self.data[:25000]
                self.targets = self.targets[:25000]
            else:
                # training data for shadow model
                idx_shadow = np.arange(25000,50000) 
                np.random.shuffle(idx_shadow) #in place 操作，没有返回值
                start=2500*(num%10) # 原来的代码中是 2500*num%10,计算逻辑错误
                self.data = self.data[idx_shadow[start:start+2500]]
                self.targets = self.targets[idx_shadow[start:start+2500]]
        else:
            if target:
                # test data for target model
                self.data = self.data[:5000]
                self.targets = self.targets[:5000]
            else:
                # test data for shadow model
                idx_shadow = np.arange(5000,10000) 
                np.random.shuffle(idx_shadow) # in place
                start=500*(num%10)
                self.data = self.data[idx_shadow[start: start+2500]]
                self.targets = self.targets[idx_shadow[start: start+2500]]

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            if self.target:
                index = index % 25000
            else:
                index = index % 2500
        else:
            if self.target:
                index = index % 5000
            else:
                index = index % 500
        img, target = self.data[index], int(self.targets[index])

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target
    
def get_data_for_final_eval(models, all_dataloaders, device):
    Y = []
    X = []
    C = []
    for idx_model, model in enumerate(models):
        model.eval()
        #print(all_dataloaders)
        dataloaders = all_dataloaders[idx_model]
        for phase in ['train', 'val']:
            for batch_idx, (data, target) in enumerate(dataloaders[phase]):
                inputs, labels = data.to(device), target.to(device)
                output = model(inputs)
                for out in output.cpu().detach().numpy():
                    X.append(out)
                    if phase == "train":
                        Y.append(1)
                    else:
                        Y.append(0)
                for cla in labels.cpu().detach().numpy():
                    C.append(cla)
    return (np.array(X), np.array(Y), np.array(C))

<h3>train model and return data for MIA</h3>

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import tqdm
import copy


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25):
    print("DATASET SIZE", dataset_sizes)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0
    return_value_train = np.zeros((4,num_epochs))

    for epoch in range(num_epochs):
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print('-' * 10)

        X = [] ## The feature of the final logits for MIA classifier
        Y = [] ## 1:"in" training sample 0:"out" training sample

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                if scheduler is not None:
                    scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data, target in tqdm.tqdm(dataloaders[phase]):
                inputs, labels = data.to(device), target.to(device)


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    ##### Generate data for MIA on the final epoch, return X and Y
                    ##### X is models output 
                    ##### Y denotes whether such sample is used for training
                    ##### Point 2


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                print(epoch, best_acc)

    return model, best_acc, np.array(X), np.array(Y)


<h3>Train target model and use target model's training set and validation set as validation set for MIA</h3>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import *
from torch.optim import lr_scheduler
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


data_train_target = custum_MNIST(True, 0, '../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
data_test_target = custum_MNIST(True, 0, '../data', train=False, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ]))

criterion = nn.CrossEntropyLoss()
train_loader_target = torch.utils.data.DataLoader(data_train_target, batch_size=64, shuffle=True)
test_loader_target = torch.utils.data.DataLoader(data_test_target, batch_size=64, shuffle=True)
dataloaders_target = {"train": train_loader_target, "val": test_loader_target}
dataset_sizes_target = {"train": len(data_train_target), "val": len(data_test_target)}
print("TAILLE dataset", dataset_sizes_target)
model_target = Net_mnist().to(device)
optimizer = optim.SGD(model_target.parameters(), lr=0.001, momentum=0.5)
exp_lr_scheduler = None

model_target, best_acc_target, data_test_set, label_test_set = train(model_target, criterion, optimizer, exp_lr_scheduler, dataloaders_target, dataset_sizes_target,
                       num_epochs=40)

print(best_acc_target)

<h3>Train shadow model and use shadow model's training set and validation set as training set for MIA</h3>

In [ ]:
data_train_set = []
label_train_set = []

for num_model_shadow in range(20):
    ############# Train 20 shadow models each 40 epochs and get their output dataset and label_set for MIA
    ############# Store them in data_train_set and label_train_set
    ############# Shadow model can also use Net_mnist
    ############# The training optimization setting and scheduler can following the target model's setting
    ############# Point 5

print("Finished Shadow Model Training")

<h3>Process datasets</h3>

In [22]:
data_train_set = np.concatenate(data_train_set)
label_train_set = np.concatenate(label_train_set)
data_train_set, label_train_set = shuffle(data_train_set, label_train_set, random_state=42)
data_test_set, label_test_set = shuffle(data_test_set, label_test_set, random_state=42)
print("Finished Shadow Dataset Generation")

Finished Shadow Dataset Generation


<h3>Use lightgbm to attack</h3>

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(objective='binary', reg_lambda=10, n_estimators=10000)
print("Start Fit")
model.fit(data_train_set, label_train_set)
print("Finish Fit")
y_pred_lgbm = model.predict(data_test_set)

<h3>Get acc, should larger than 85%</h3>

In [ ]:
print((y_pred_lgbm==label_test_set).sum()/(label_test_set).sum())